In [107]:
import pandas as pd
import numpy as np
import yfinance as yf
from pypfopt import risk_models

from pypfopt.efficient_frontier import EfficientFrontier

from pypfopt import expected_returns


In [56]:
data = pd.read_csv('./master_forecasts.csv', index_col='Date_Forecast')
data

,JNJ,GS,RIO,AAPL,JPM,BHP,DIS,XOM,CVX,AMZN,BA,MSFT,NEE,NVDA,CAT,MCD,NVS
Date_Forecast,,,,,,,,,,,,,,,,,
2007-12-21,0.023264,0.045352,0.057726,0.083394,0.036417,0.052193,0.023608,0.063423,0.032575,0.037432,0.055325,0.046126,0.026180,0.054642,0.042926,0.032685,0.025529
2007-12-28,0.022341,0.058122,0.045782,0.046853,0.035109,0.040304,0.024848,0.028423,0.026072,0.086054,0.030517,0.025668,0.031470,0.056102,0.036753,0.039631,0.022063
2008-01-04,0.069348,0.053316,0.064487,0.044804,0.031469,0.045631,0.021916,0.026210,0.041220,0.053054,0.041650,0.033363,0.030850,0.054798,0.044424,0.032147,0.028862
2008-01-11,0.020626,0.037606,0.045752,0.045409,0.031678,0.043211,0.023816,0.034866,0.038515,0.054489,0.041594,0.028024,0.028475,0.050171,0.039732,0.025847,0.023815
2008-01-18,0.034864,0.042194,0.069117,0.049226,0.040363,0.052146,0.036402,0.031222,0.029583,0.031774,0.029274,0.026534,0.075412,0.050569,0.056783,0.052350,0.036230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-20,0.033833,0.026500,0.051083,0.041998,0.030983,0.060443,0.030774,0.034371,0.056785,0.043167,0.030228,0.034039,0.028050,0.055507,0.036039,0.024783,0.035350
2015-11-27,0.031334,0.034626,0.049554,0.037472,0.025606,0.055037,0.026694,0.032837,0.031077,0.056929,0.058610,0.054790,0.028886,0.049952,0.032945,0.035115,0.030087
2015-12-04,0.033112,0.038427,0.044371,0.033719,0.026229,0.054007,0.036061,0.032583,0.041884,0.020785,0.031226,0.036495,0.045193,0.054145,0.032817,0.025907,0.035360


In [34]:
stocks = ['AAPL', 'MSFT', 'NVDA', 'JNJ', 'NVS','JPM','GS','AMZN','DIS','MCD','NEE','BA','CAT','XOM','CVX','RIO','BHP']

start_date = '2009-01-01' 

stock_data = yf.download(stocks, start=start_date)['Close']
stock_data = stock_data.dropna()
stock_data = stock_data.reindex(columns=stocks)
stock_prices = stock_data[stocks].values



[*********************100%***********************]  17 of 17 completed


In [83]:
date = '2015-12-04'

In [58]:
window_length = 650
data_window = stock_data.loc[:date].tail(window_length)
data_window

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
Date,,,,,,,,,,,,,,,,,
2013-05-09,16.313213,32.660000,3.4775,85.150002,66.505379,49.040001,148.539993,13.008000,66.669998,99.690002,19.959999,94.610001,89.949997,91.029999,123.639999,47.330002,62.149864
2013-05-10,16.177500,32.689999,3.6350,85.760002,67.150536,48.959999,149.100006,13.181500,67.199997,100.199997,20.000000,94.239998,88.620003,90.139999,123.230003,46.970001,61.784122
2013-05-13,16.240713,33.029999,3.5600,85.849998,66.890678,49.669998,149.630005,13.225500,67.320000,100.379997,19.905001,94.760002,88.070000,90.099998,122.849998,46.209999,60.972347
2013-05-14,15.852143,33.529999,3.6125,86.970001,66.836922,50.230000,154.520004,13.416500,67.470001,101.199997,20.047501,96.110001,87.550003,91.110001,125.000000,45.330002,60.900982
2013-05-15,15.316071,33.849998,3.6750,87.639999,67.652328,51.090000,155.610001,13.328000,67.669998,101.949997,20.290001,97.019997,87.000000,91.239998,123.010002,44.380001,59.857269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-30,29.575001,54.349998,7.9300,101.239998,76.379929,66.680000,190.020004,33.240002,113.470001,114.160004,24.965000,145.449997,72.650002,81.660004,91.320000,33.080002,23.800179
2015-12-01,29.334999,55.220001,8.1875,102.360001,77.016129,67.610001,193.070007,33.952999,115.389999,114.449997,25.142500,147.740005,71.559998,81.889999,92.480003,33.400002,24.362177
2015-12-02,29.070000,55.209999,8.1250,102.050003,76.182793,66.660004,190.259995,33.800499,114.000000,113.720001,24.497499,146.960007,71.029999,79.550003,90.250000,32.480000,23.987511


In [81]:
import os
# Define your file path
File_path = "../Output/assets/Drop000/"
Files = os.listdir(File_path)

def retrieve_var(prediction= 'Forecast_T_ANN_ARCH'):
    # Initialize an empty DataFrame to store forecasts
    Forecasts = pd.DataFrame()

    # Loop through all files
    for file in Files:
        # Skip the ".DS_Store" file
        if file == ".DS_Store":
            continue

        path = os.path.join(File_path, file)
        asset_name = file.split('.')[0].split('_')[3]  # Assumes file name is the asset name

        asset = pd.read_csv(path)
        
        # Create a DataFrame with the asset names as columns and predicted variances as the values
        df = pd.DataFrame(asset[prediction].values, columns=[asset_name])
        df['Date_Forecast'] = asset['Date_Forecast']

        # If Forecasts DataFrame is empty, assign df to it. Else, merge the new df with Forecasts on 'Date_Forecast'
        if Forecasts.empty:
            Forecasts = df
        else:
            Forecasts = pd.merge(Forecasts, df, how='outer', on='Date_Forecast')

    # Set 'Date_Forecast' as the index
    Forecasts.set_index('Date_Forecast', inplace=True)
    return Forecasts

# Save the DataFrame to CSV
data = retrieve_var('Forecast_T_ANN_ARCH')

In [87]:
pred = data[data.index == date]
pred 

,JNJ,GS,RIO,AAPL,JPM,BHP,DIS,XOM,CVX,AMZN,BA,MSFT,NEE,NVDA,CAT,MCD,NVS
Date_Forecast,,,,,,,,,,,,,,,,,
2015-12-04,0.033112,0.038427,0.044371,0.033719,0.026229,0.054007,0.036061,0.032583,0.041884,0.020785,0.031226,0.036495,0.045193,0.054145,0.032817,0.025907,0.03536


## Get returns then covariance

In [85]:
cov = risk_models.sample_cov(data_window)
cov


,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
AAPL,0.058274,0.018206,0.019168,0.010635,0.010151,0.015689,0.015532,0.018577,0.013300,0.010370,0.006528,0.016455,0.016884,0.012073,0.012828,0.016721,0.017755
MSFT,0.018206,0.062191,0.025893,0.013602,0.012105,0.021510,0.020994,0.027030,0.017956,0.014579,0.011996,0.015645,0.019634,0.016352,0.017380,0.019693,0.024652
NVDA,0.019168,0.025893,0.080974,0.012013,0.012549,0.023663,0.023516,0.023010,0.021142,0.012394,0.009928,0.016939,0.017510,0.017696,0.018298,0.020796,0.020378
JNJ,0.010635,0.013602,0.012013,0.022835,0.010566,0.015462,0.015572,0.015778,0.013759,0.012092,0.011097,0.014879,0.011858,0.013271,0.013140,0.012845,0.014146
NVS,0.010151,0.012105,0.012549,0.010566,0.026887,0.012448,0.013018,0.012610,0.013560,0.008817,0.009904,0.011366,0.008999,0.010747,0.011320,0.016020,0.016725
JPM,0.015689,0.021510,0.023663,0.015462,0.012448,0.038988,0.031055,0.022781,0.019888,0.013637,0.011010,0.019604,0.018901,0.019105,0.020044,0.021848,0.021244
GS,0.015532,0.020994,0.023516,0.015572,0.013018,0.031055,0.038817,0.024616,0.020239,0.013608,0.010162,0.020024,0.019901,0.018880,0.019726,0.022448,0.021558
AMZN,0.018577,0.027030,0.023010,0.015778,0.012610,0.022781,0.024616,0.098391,0.023871,0.015277,0.009997,0.021376,0.018288,0.015854,0.017691,0.020650,0.025167
DIS,0.013300,0.017956,0.021142,0.013759,0.013560,0.019888,0.020239,0.023871,0.038648,0.013231,0.010704,0.018475,0.013951,0.014461,0.014461,0.017491,0.017818
MCD,0.010370,0.014579,0.012394,0.012092,0.008817,0.013637,0.013608,0.015277,0.013231,0.024089,0.009396,0.012998,0.012230,0.012304,0.011611,0.013629,0.014545


In [88]:
data_window.cov()

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
AAPL,31.923951,26.500989,4.179735,24.152525,48.025130,22.168848,82.644630,14.564173,92.378426,1.428091,10.742626,64.376203,-13.998206,-18.704637,-50.266580,-17.660918,-49.122726
MSFT,26.500989,31.706264,4.534323,27.164955,38.444865,21.352706,69.328501,15.938426,83.256025,6.887645,10.033538,50.883926,-3.940447,-11.200078,-42.112469,-14.766887,-43.453829
NVDA,4.179735,4.534323,0.910850,3.101053,5.605856,3.378175,9.928189,3.599164,14.154481,2.257973,1.508151,9.393243,-3.848011,-3.572147,-9.435295,-3.628059,-8.811014
JNJ,24.152525,27.164955,3.101053,38.499787,39.954663,15.562276,56.756696,4.070929,65.336864,0.666195,11.088731,48.865845,26.254710,9.507337,-6.738079,1.527134,-17.372857
NVS,48.025130,38.444865,5.605856,39.954663,81.263243,33.095283,124.007038,17.566931,139.411944,-2.476264,17.808065,99.595311,-6.031933,-21.918114,-69.635932,-19.484564,-64.850171
JPM,22.168848,21.352706,3.378175,15.562276,33.095283,22.766519,71.859989,16.306347,74.357944,5.002669,7.292965,47.610921,-14.880620,-17.329083,-46.222047,-16.534536,-42.475941
GS,82.644630,69.328501,9.928189,56.756696,124.007038,71.859989,267.752071,42.808951,248.569112,0.488280,25.420720,163.303170,-49.616225,-57.903185,-141.841616,-54.759993,-140.527075
AMZN,14.564173,15.938426,3.599164,4.070929,17.566931,16.306347,42.808951,22.003323,57.750828,12.203281,4.193738,41.635124,-31.312892,-22.930182,-52.230410,-19.425752,-42.385214
DIS,92.378426,83.256025,14.154481,65.336864,139.411944,74.357944,248.569112,57.750828,298.117927,19.080090,31.928642,196.591897,-57.091942,-65.963360,-176.589800,-62.415219,-163.587544
MCD,1.428091,6.887645,2.257973,0.666195,-2.476264,5.002669,0.488280,12.203281,19.080090,20.052513,0.537334,11.753048,-13.864763,-5.703741,-14.141716,-10.656012,-15.715534


In [64]:

# replace diagonal elements of cov matrix by GARCH-predicted variance.
# GARCH_var = GARCH_predict(symbol_list, start, end, interval)
cov =  risk_models.sample_cov(data_window)
adjust_cov_matrix =  cov.copy()
for symbols in stocks:
    adjust_cov_matrix[symbols][symbols] = pred[symbols]

In [66]:
cov

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
AAPL,0.058274,0.018206,0.019168,0.010635,0.010151,0.015689,0.015532,0.018577,0.013300,0.010370,0.006528,0.016455,0.016884,0.012073,0.012828,0.016721,0.017755
MSFT,0.018206,0.062191,0.025893,0.013602,0.012105,0.021510,0.020994,0.027030,0.017956,0.014579,0.011996,0.015645,0.019634,0.016352,0.017380,0.019693,0.024652
NVDA,0.019168,0.025893,0.080974,0.012013,0.012549,0.023663,0.023516,0.023010,0.021142,0.012394,0.009928,0.016939,0.017510,0.017696,0.018298,0.020796,0.020378
JNJ,0.010635,0.013602,0.012013,0.022835,0.010566,0.015462,0.015572,0.015778,0.013759,0.012092,0.011097,0.014879,0.011858,0.013271,0.013140,0.012845,0.014146
NVS,0.010151,0.012105,0.012549,0.010566,0.026887,0.012448,0.013018,0.012610,0.013560,0.008817,0.009904,0.011366,0.008999,0.010747,0.011320,0.016020,0.016725
JPM,0.015689,0.021510,0.023663,0.015462,0.012448,0.038988,0.031055,0.022781,0.019888,0.013637,0.011010,0.019604,0.018901,0.019105,0.020044,0.021848,0.021244
GS,0.015532,0.020994,0.023516,0.015572,0.013018,0.031055,0.038817,0.024616,0.020239,0.013608,0.010162,0.020024,0.019901,0.018880,0.019726,0.022448,0.021558
AMZN,0.018577,0.027030,0.023010,0.015778,0.012610,0.022781,0.024616,0.098391,0.023871,0.015277,0.009997,0.021376,0.018288,0.015854,0.017691,0.020650,0.025167
DIS,0.013300,0.017956,0.021142,0.013759,0.013560,0.019888,0.020239,0.023871,0.038648,0.013231,0.010704,0.018475,0.013951,0.014461,0.014461,0.017491,0.017818
MCD,0.010370,0.014579,0.012394,0.012092,0.008817,0.013637,0.013608,0.015277,0.013231,0.024089,0.009396,0.012998,0.012230,0.012304,0.011611,0.013629,0.014545


In [65]:
adjust_cov_matrix

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
AAPL,0.042332,0.018206,0.019168,0.010635,0.010151,0.015689,0.015532,0.018577,0.013300,0.010370,0.006528,0.016455,0.016884,0.012073,0.012828,0.016721,0.017755
MSFT,0.018206,0.031458,0.025893,0.013602,0.012105,0.021510,0.020994,0.027030,0.017956,0.014579,0.011996,0.015645,0.019634,0.016352,0.017380,0.019693,0.024652
NVDA,0.019168,0.025893,0.048492,0.012013,0.012549,0.023663,0.023516,0.023010,0.021142,0.012394,0.009928,0.016939,0.017510,0.017696,0.018298,0.020796,0.020378
JNJ,0.010635,0.013602,0.012013,0.012198,0.010566,0.015462,0.015572,0.015778,0.013759,0.012092,0.011097,0.014879,0.011858,0.013271,0.013140,0.012845,0.014146
NVS,0.010151,0.012105,0.012549,0.010566,0.016388,0.012448,0.013018,0.012610,0.013560,0.008817,0.009904,0.011366,0.008999,0.010747,0.011320,0.016020,0.016725
JPM,0.015689,0.021510,0.023663,0.015462,0.012448,0.039175,0.031055,0.022781,0.019888,0.013637,0.011010,0.019604,0.018901,0.019105,0.020044,0.021848,0.021244
GS,0.015532,0.020994,0.023516,0.015572,0.013018,0.031055,0.046471,0.024616,0.020239,0.013608,0.010162,0.020024,0.019901,0.018880,0.019726,0.022448,0.021558
AMZN,0.018577,0.027030,0.023010,0.015778,0.012610,0.022781,0.024616,0.035347,0.023871,0.015277,0.009997,0.021376,0.018288,0.015854,0.017691,0.020650,0.025167
DIS,0.013300,0.017956,0.021142,0.013759,0.013560,0.019888,0.020239,0.023871,0.023975,0.013231,0.010704,0.018475,0.013951,0.014461,0.014461,0.017491,0.017818
MCD,0.010370,0.014579,0.012394,0.012092,0.008817,0.013637,0.013608,0.015277,0.013231,0.008839,0.009396,0.012998,0.012230,0.012304,0.011611,0.013629,0.014545


## 
for monthly re-balancing, i should be using the 1 month ahead predicted volatility.
1. investigate the GARCH scaling
2. how i handle the weekly resampling in the predictions
3. Ensure correct volatility for the frequency of interest.
   
 $VOL=  \sigma \sqrt{T}$
+ v = volatility over some interval of time
+ σ =standard deviation of returns
+ T = number of periods in the time horizon

volatiltiy = variance^{1/2} = standard deviation

### Testing GARCH forecasts

In [43]:
from datetime import date, datetime, timedelta
from arch import arch_model

In [44]:
start_date = '2009-01-01' 
end = '2013-01-01'
returns = yf.download(stocks, start_date, end).pct_change()['Adj Close'].dropna()

[*********************100%***********************]  17 of 17 completed


In [45]:
Data = pd.DataFrame()
Data['Daily_Returns'] = returns['AAPL'].copy()

In [1]:
def FIG_GARCH(Data):
    AR_Data=Data['Daily_Returns']*100
    FIGARCH11 = arch_model(AR_Data, dist ='t', vol="FIGARCH")
    res = FIGARCH11.fit(update_freq=5)
    forecasts = res.forecast(horizon=2, method='simulation', simulations=1000)
    print(forecasts.variance.dropna().tail())
    
FIG_GARCH(Data)

    

NameError: name 'Data' is not defined

In [48]:
#Fitting of FIGARCH11(1,1)
def FIGARCH_Model_Student(Data,hor=1):
    AR_Data=Data['Daily_Returns']*100
    FIGARCH11 = arch_model(AR_Data, dist ='t', vol="FIGARCH")
    res_FIGARCH11 = FIGARCH11.fit(disp='off', update_freq=5)
    CV_FIGARCH11 = res_FIGARCH11.conditional_volatility
    For_CV_FIGARCH11 = res_FIGARCH11.forecast(horizon=1, method='simulation', simulations=1000).variance.dropna()
    return FIGARCH11, res_FIGARCH11, CV_FIGARCH11, For_CV_FIGARCH11

In [49]:
FIGARCH11, res_FIGARCH11, CV_FIGARCH11, For_CV_FIGARCH11 = FIGARCH_Model_Student(Data,1)


/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


In [79]:
For_CV_FIGARCH11/100


0.045545036114844686

In [80]:
np.var(Data)

Daily_Returns    0.000336
dtype: float64

In [50]:
#Fitting of GARCH(1,1)
def GARCH_Model_Student (Data):
    AR_Data=Data['Daily_Returns']*100
    GARCH11 = arch_model(AR_Data, dist ='t')
    res_GARCH11 = GARCH11.fit(disp='off')
    CV_GARCH11 = res_GARCH11.conditional_volatility
    For_CV_GARCH11 = np.array(res_GARCH11.forecast(horizon=1).variance.dropna())[0][0]
    return GARCH11, res_GARCH11, CV_GARCH11, For_CV_GARCH11


In [54]:
FIGARCH11, res_FIGARCH11, CV_FIGARCH11, For_CV_FIGARCH11 = GARCH_Model_Student(Data)

/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


In [55]:
CV_FIGARCH11

Date
2009-01-05    2.950163
2009-01-06    3.003349
2009-01-07    2.914811
2009-01-08    2.856440
2009-01-09    2.769503
                ...   
2012-12-24    2.133035
2012-12-26    2.055234
2012-12-27    2.022117
2012-12-28    1.951550
2012-12-31    1.911718
Name: cond_vol, Length: 1005, dtype: float64

### Attempt to use our weekly forecasts.
+ I think we need to address teh frequency

if i feed in weekly returns, then i can forecast 4 time step ahead, and take an average for the monthly volatility. 
2. It will need to move at a monthly time step, (not daily)
3. Take the the history weekly returns and fit, predict the next 4.

## Case 1
a it is variance, it is daily ( $* T(5*4))$

b it is variance already weekly (*T(4) )

c it is SD and daily  (\sqrt{T(5*4)})

d it is SD and weekly (\sqrt{T(4)})

In [160]:
import pandas as pd
import numpy as np
import yfinance as yf
from arch import arch_model

# Define the tickers of the stocks you're interested in
tickers = ['AAPL', 'MSFT', 'NVDA', 'JNJ', 'NVS','JPM','GS','AMZN','DIS','MCD','NEE','BA','CAT','XOM','CVX','RIO','BHP']

# Define the start and end dates for the data
start_date = '2009-01-01' 
end_date = '2022-12-31'

# Initialize an empty DataFrame to store the results
volatility_df = pd.DataFrame()

# Loop over each ticker
for ticker in tickers:
    # Download the stock price data
    data = yf.download(ticker, start=start_date, end=end_date)
    
   
    # Calculate the daily returns
    data['returns'] = data['Adj Close'].pct_change()
    
    # Drop missing values
    data = data.dropna()

    # Define the GARCH model
    model = arch_model(data['returns'], vol='Garch', p=1, q=1)

    # Fit the GARCH model
    res = model.fit(disp='off')

    # Forecast the volatility
    forecasts = res.forecast(start=0)

    # Add the forecasted volatility to the DataFrame
    volatility_df[ticker] = np.sqrt(forecasts.variance['h.1']) * np.sqrt(252)

# Drop missing values from the DataFrame
volatility_df = volatility_df.dropna()

# Print the DataFrame
print(volatility_df)


[*********************100%***********************]  1 of 1 completed


/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003357. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


[*********************100%***********************]  1 of 1 completed


/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002875. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


[*********************100%***********************]  1 of 1 completed


...

/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003026. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


[*********************100%***********************]  1 of 1 completed


/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.000596. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


[*********************100%***********************]  1 of 1 completed
                AAPL      MSFT      NVDA       JNJ       NVS       JPM  \
Date                                                                     
2009-01-05  0.476896  0.550362  0.729977  0.197488  0.268087  1.238276   
2009-01-06  0.466240  0.533433  0.715909  0.189684  0.260526  1.167202   
2009-01-07  0.459032  0.559907  0.727448  0.186365  0.245406  1.137966   
2009-01-08  0.448588  0.551764  0.711221  0.176873  0.245225  1.080677   
2009-01-09  0.443230  0.544658  0.718941  0.167631  0.272596  1.042005   
...              ...       ...       ...       ...       ...       ...   
2022-12-23  0.337600  0.314669  0.547377  0.128221  0.150673  0.217768   
2022-12-27  0.332794  0.307335  0.592088  0.122705  0.147657  0.210566   
2022-12-28  0.343394  0.301539  0.575333  0.120122  0.145976  0.204848   
2022-12-29  0.346811  0.307989  0.574996  0.117398  0.148605  0.199838   
2022-12-30  0.336983  0.300218  0.558276  0

/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0004922. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


In [185]:
volatility_df

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
Date,,,,,,,,,,,,,,,,,
2009-01-05,0.476896,0.550362,0.729977,0.197488,0.268087,1.238276,0.962969,0.650271,0.487940,0.278631,0.256733,0.433030,0.571603,0.311552,0.370289,1.014954,0.715734
2009-01-06,0.466240,0.533433,0.715909,0.189684,0.260526,1.167202,0.904523,0.673319,0.487137,0.282192,0.254265,0.409635,0.553587,0.306201,0.352040,1.224725,0.723809
2009-01-07,0.459032,0.559907,0.727448,0.186365,0.245406,1.137966,0.883382,0.638562,0.500192,0.278754,0.247821,0.424829,0.561545,0.317472,0.400617,1.212027,0.779500
2009-01-08,0.448588,0.551764,0.711221,0.176873,0.245225,1.080677,0.831496,0.602266,0.486019,0.273789,0.237738,0.401896,0.543946,0.304392,0.378157,1.138304,0.734207
2009-01-09,0.443230,0.544658,0.718941,0.167631,0.272596,1.042005,0.786605,0.583590,0.479553,0.266995,0.251059,0.383062,0.531003,0.304239,0.370472,1.138869,0.698835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,0.337600,0.314669,0.547377,0.128221,0.150673,0.217768,0.239513,0.374497,0.367777,0.164467,0.208343,0.374487,0.270156,0.258593,0.279805,0.254076,0.263946
2022-12-27,0.332794,0.307335,0.592088,0.122705,0.147657,0.210566,0.235734,0.381079,0.363283,0.161378,0.198366,0.355130,0.268132,0.254357,0.272004,0.247839,0.258010
2022-12-28,0.343394,0.301539,0.575333,0.120122,0.145976,0.204848,0.226668,0.371191,0.364465,0.160029,0.196489,0.338729,0.268754,0.255713,0.269503,0.246646,0.260720


In [162]:
import numpy as np



# Replace infinities with NaN
# returns.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop NaN values
volatility_df.dropna(inplace=True)

# Or fill forward
# volatility_df.fillna(method='ffill', inplace=True)

# Or fill backward
# volatility_df.fillna(method='bfill', inplace=True)

# Or interpolate
volatility_df.interpolate(method='linear', inplace=True)


In [163]:
month_vol = volatility_df*np.sqrt(252)
month_var = month_vol**2
month_var

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
Date,,,,,,,,,,,,,,,,,
2009-01-05,57.312220,76.330434,134.282318,9.828338,18.111340,386.398344,233.681848,106.558896,59.997478,19.564074,16.609762,47.253790,82.335853,24.460248,34.552765,259.593130,129.093309
2009-01-06,54.779654,71.706892,129.156529,9.066933,17.104181,343.314718,206.176568,114.246170,59.800118,20.067367,16.291932,42.285752,77.227646,23.627357,31.230836,377.987493,132.022765
2009-01-07,53.098916,79.001093,133.353357,8.752408,15.176480,326.331311,196.651450,102.756033,63.048419,19.581363,15.476620,45.480892,79.463788,25.398720,40.444397,370.190361,153.120424
2009-01-08,50.710203,76.719630,127.470572,7.883545,15.154156,294.301625,174.229183,91.406405,59.526068,18.889973,14.242916,40.703152,74.560996,23.348919,36.036777,326.525554,135.843154
2009-01-09,49.506165,74.756516,130.252932,7.081250,18.725746,273.615160,155.924333,85.825338,57.952754,17.964183,15.883664,36.977528,71.054969,23.325409,34.586926,326.849622,123.069349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,28.721374,24.952170,75.504767,4.143050,5.720999,11.950544,14.456314,35.342556,34.085484,6.816474,10.938468,35.340569,18.392026,16.851296,19.729290,16.267778,17.556143
2022-12-27,27.909465,23.802625,88.343246,3.794267,5.494216,11.173169,14.003782,36.595807,33.257604,6.562768,9.915943,31.781548,18.117535,16.303735,18.644539,15.478845,16.775458
2022-12-28,29.715719,22.913339,83.414153,3.636160,5.369890,10.574633,12.947362,34.721250,33.474280,6.453536,9.729173,28.913886,18.201615,16.478016,18.303165,15.330291,17.129723


In [184]:
volatility_df.to_csv('test_vol.csv')

In [165]:
volatility_df

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
Date,,,,,,,,,,,,,,,,,
2009-01-05,0.476896,0.550362,0.729977,0.197488,0.268087,1.238276,0.962969,0.650271,0.487940,0.278631,0.256733,0.433030,0.571603,0.311552,0.370289,1.014954,0.715734
2009-01-06,0.466240,0.533433,0.715909,0.189684,0.260526,1.167202,0.904523,0.673319,0.487137,0.282192,0.254265,0.409635,0.553587,0.306201,0.352040,1.224725,0.723809
2009-01-07,0.459032,0.559907,0.727448,0.186365,0.245406,1.137966,0.883382,0.638562,0.500192,0.278754,0.247821,0.424829,0.561545,0.317472,0.400617,1.212027,0.779500
2009-01-08,0.448588,0.551764,0.711221,0.176873,0.245225,1.080677,0.831496,0.602266,0.486019,0.273789,0.237738,0.401896,0.543946,0.304392,0.378157,1.138304,0.734207
2009-01-09,0.443230,0.544658,0.718941,0.167631,0.272596,1.042005,0.786605,0.583590,0.479553,0.266995,0.251059,0.383062,0.531003,0.304239,0.370472,1.138869,0.698835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,0.337600,0.314669,0.547377,0.128221,0.150673,0.217768,0.239513,0.374497,0.367777,0.164467,0.208343,0.374487,0.270156,0.258593,0.279805,0.254076,0.263946
2022-12-27,0.332794,0.307335,0.592088,0.122705,0.147657,0.210566,0.235734,0.381079,0.363283,0.161378,0.198366,0.355130,0.268132,0.254357,0.272004,0.247839,0.258010
2022-12-28,0.343394,0.301539,0.575333,0.120122,0.145976,0.204848,0.226668,0.371191,0.364465,0.160029,0.196489,0.338729,0.268754,0.255713,0.269503,0.246646,0.260720


In [166]:
mu = expected_returns.mean_historical_return(data_window)
S =risk_models.sample_cov(data_window)


In [167]:
volatility_df[volatility_df.index == date]

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
Date,,,,,,,,,,,,,,,,,
2015-12-04,0.266396,0.260437,0.418262,0.169985,0.183653,0.262498,0.269413,0.280977,0.24273,0.189539,0.213464,0.221155,0.26113,0.236009,0.262434,0.295345,0.340108


In [191]:
S =  risk_models.sample_cov(data_window)
# S =np.cov(data_window)
S_adj =  S.copy()
for symbols in stocks:
    S_adj[symbols][symbols] = volatility_df[volatility_df.index == date][symbols]**(2) #Volatility = Variance^1/2 = Standard Deviation

In [192]:
S_adj

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
AAPL,0.070967,0.018206,0.019168,0.010635,0.010151,0.015689,0.015532,0.018577,0.013300,0.010370,0.006528,0.016455,0.016884,0.012073,0.012828,0.016721,0.017755
MSFT,0.018206,0.067827,0.025893,0.013602,0.012105,0.021510,0.020994,0.027030,0.017956,0.014579,0.011996,0.015645,0.019634,0.016352,0.017380,0.019693,0.024652
NVDA,0.019168,0.025893,0.174943,0.012013,0.012549,0.023663,0.023516,0.023010,0.021142,0.012394,0.009928,0.016939,0.017510,0.017696,0.018298,0.020796,0.020378
JNJ,0.010635,0.013602,0.012013,0.028895,0.010566,0.015462,0.015572,0.015778,0.013759,0.012092,0.011097,0.014879,0.011858,0.013271,0.013140,0.012845,0.014146
NVS,0.010151,0.012105,0.012549,0.010566,0.033728,0.012448,0.013018,0.012610,0.013560,0.008817,0.009904,0.011366,0.008999,0.010747,0.011320,0.016020,0.016725
JPM,0.015689,0.021510,0.023663,0.015462,0.012448,0.068905,0.031055,0.022781,0.019888,0.013637,0.011010,0.019604,0.018901,0.019105,0.020044,0.021848,0.021244
GS,0.015532,0.020994,0.023516,0.015572,0.013018,0.031055,0.072584,0.024616,0.020239,0.013608,0.010162,0.020024,0.019901,0.018880,0.019726,0.022448,0.021558
AMZN,0.018577,0.027030,0.023010,0.015778,0.012610,0.022781,0.024616,0.078948,0.023871,0.015277,0.009997,0.021376,0.018288,0.015854,0.017691,0.020650,0.025167
DIS,0.013300,0.017956,0.021142,0.013759,0.013560,0.019888,0.020239,0.023871,0.058918,0.013231,0.010704,0.018475,0.013951,0.014461,0.014461,0.017491,0.017818
MCD,0.010370,0.014579,0.012394,0.012092,0.008817,0.013637,0.013608,0.015277,0.013231,0.035925,0.009396,0.012998,0.012230,0.012304,0.011611,0.013629,0.014545


In [193]:
S 

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
AAPL,0.058274,0.018206,0.019168,0.010635,0.010151,0.015689,0.015532,0.018577,0.013300,0.010370,0.006528,0.016455,0.016884,0.012073,0.012828,0.016721,0.017755
MSFT,0.018206,0.062191,0.025893,0.013602,0.012105,0.021510,0.020994,0.027030,0.017956,0.014579,0.011996,0.015645,0.019634,0.016352,0.017380,0.019693,0.024652
NVDA,0.019168,0.025893,0.080974,0.012013,0.012549,0.023663,0.023516,0.023010,0.021142,0.012394,0.009928,0.016939,0.017510,0.017696,0.018298,0.020796,0.020378
JNJ,0.010635,0.013602,0.012013,0.022835,0.010566,0.015462,0.015572,0.015778,0.013759,0.012092,0.011097,0.014879,0.011858,0.013271,0.013140,0.012845,0.014146
NVS,0.010151,0.012105,0.012549,0.010566,0.026887,0.012448,0.013018,0.012610,0.013560,0.008817,0.009904,0.011366,0.008999,0.010747,0.011320,0.016020,0.016725
JPM,0.015689,0.021510,0.023663,0.015462,0.012448,0.038988,0.031055,0.022781,0.019888,0.013637,0.011010,0.019604,0.018901,0.019105,0.020044,0.021848,0.021244
GS,0.015532,0.020994,0.023516,0.015572,0.013018,0.031055,0.038817,0.024616,0.020239,0.013608,0.010162,0.020024,0.019901,0.018880,0.019726,0.022448,0.021558
AMZN,0.018577,0.027030,0.023010,0.015778,0.012610,0.022781,0.024616,0.098391,0.023871,0.015277,0.009997,0.021376,0.018288,0.015854,0.017691,0.020650,0.025167
DIS,0.013300,0.017956,0.021142,0.013759,0.013560,0.019888,0.020239,0.023871,0.038648,0.013231,0.010704,0.018475,0.013951,0.014461,0.014461,0.017491,0.017818
MCD,0.010370,0.014579,0.012394,0.012092,0.008817,0.013637,0.013608,0.015277,0.013231,0.024089,0.009396,0.012998,0.012230,0.012304,0.011611,0.013629,0.014545


In [194]:
ef = EfficientFrontier(mu, S_adj, weight_bounds=(0, 1))  # Set bounds to allow shorting
raw_weights = ef.max_sharpe()  # Optimize for maximal Sharpe ratio
cleaned_weights = ef.clean_weights()  #

We learned of annulaized volatility, that the riskmodels.sample_cov assumes. 
I need to set it to monthly now, as well as the forecasted volatilities.
i need to check how that rebalancing loop works, i think its messing up.

In [ ]:
# Set the number of assets, assumed that 'df' is your DataFrame with historical prices.
df = stock_data.copy()
num_assets = len(df.columns)
stocks = df.columns

# Cred portfolio weights over time
optimal_weights_df = pd.DataFrame(index=df.index, columns=stocks)

# Set the window length for the rolling covariance calculation
window_length = 252  # Use 252 for approx one year of trading days

# Set the initial weights to be equal for all assets
initial_weights = np.array([1/num_assets]*num_assets)

for end_date in df.index[window_length:]:
    
    # Extract the window of data
    data_window = df.loc[:end_date].tail(window_length)
    
    # Calculate expected returns and covariance matrix for the window
    mu = expected_returns.mean_historical_return(data_window)

    # Get the date of the next period
    future_dates = df.index[df.index > end_date]
    if len(future_dates) > 0:
        next_period_date = future_dates[0]
    else:
        break
    # Get the date of the next period
    next_period_date = df.index[df.index > end_date][0]
    
    # Get the date of the closest week in the std_data dataframe
    closest_week_date = std_data.index[std_data.index.get_loc(next_period_date, method='nearest')]

    # Get the volatility data for the closest week
    std_data_next = std_data.loc[closest_week_date]

    
    # replace diagonal elements of cov matrix by GARCH-predicted variance.

    S =  risk_models.sample_cov(data_window)
    S_adj =  S.copy()
    for symbols in stocks:
        S_adj[symbols][symbols] = std_data_next[symbols]**(1/2) #Volatility = Variance^1/2 = Standard Deviation
    
    # Initialise and optimize the portfolio on the window of data
    ef = EfficientFrontier(mu, S_adj, weight_bounds=(0, 1))  # Set bounds to allow shorting
    raw_weights = ef.max_sharpe()  # Optimize for maximal Sharpe ratio
    cleaned_weights = ef.clean_weights()  # Clean the raw weights
    
    # Add the weights to our optimal_weights dataframe
    optimal_weights_df.loc[end_date] = [cleaned_weights.get(stock) for stock in stocks]

# Forward fill the weights for the days we aren't rebalancing
optimal_weights_df = optimal_weights_df.fillna(method='ffill')

# Print out the final DataFrame
print(optimal_weights_df.tail())

In [207]:
eom_dates = df.resample('M').last().index
eom_dates[eom_dates >= df.index[window_length]]

DatetimeIndex(['2011-08-31', '2011-09-30', '2011-10-31', '2011-11-30',
               '2011-12-31', '2012-01-31', '2012-02-29', '2012-03-31',
               '2012-04-30', '2012-05-31',
               ...
               '2022-10-31', '2022-11-30', '2022-12-31', '2023-01-31',
               '2023-02-28', '2023-03-31', '2023-04-30', '2023-05-31',
               '2023-06-30', '2023-07-31'],
              dtype='datetime64[ns]', name='Date', length=144, freq='M')

In [ ]:
# Get the end-of-month dates
eom_dates = df.resample('M').last().index

# Loop over each end-of-month date
for end_date in eom_dates[eom_dates >= df.index[window_length]]:
    
    # Extract the window of data
    data_window = df.loc[:end_date].tail(window_length)
    
    # Calculate expected returns and covariance matrix for the window
    mu = expected_returns.mean_historical_return(data_window)

    # Get the date of the next period
    future_dates = df.index[df.index > end_date]
    if len(future_dates) > 0:
        next_period_date = future_dates[0]
    else:
        break
    
    # Get the date of the closest week in the std_data dataframe
    closest_week_date = std_data.index[std_data.index.get_loc(next_period_date, method='nearest')]

    # Get the volatility data for the closest week
    std_data_next = std_data.loc[closest_week_date]

    # replace diagonal elements of cov matrix by GARCH-predicted variance.
    S =  risk_models.sample_cov(data_window)
    S_adj =  S.copy()
    for symbols in stocks:
        S_adj[symbols][symbols] = std_data_next[symbols]**(1/2) #Volatility = Variance^1/2 = Standard Deviation
    
    # Initialise and optimize the portfolio on the window of data
    ef = EfficientFrontier(mu, S_adj, weight_bounds=(0, 1))  # Set bounds to allow shorting
    raw_weights = ef.max_sharpe()  # Optimize for maximal Sharpe ratio
    cleaned_weights = ef.clean_weights()  # Clean the raw weights
    
    # Add the weights to our optimal_weights dataframe
    optimal_weights_df.loc[end_date] = [cleaned_weights.get(stock) for stock in stocks]

# Forward fill the weights for the days we aren't rebalancing
optimal_weights_df = optimal_weights_df.fillna(method='ffill')

# Print out the final DataFrame
print(optimal_weights_df.tail())
